In [1]:
%cd /teamspace/studios/this_studio

/teamspace/studios/this_studio


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [37]:
import json
import os
from pathlib import Path
from pydantic import BaseModel
from typing import Literal
import pandas as pd
import openai
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = "sk-5Z4uh0dNojtCvM-PlMOSUXtG84rStqDbAokYRKX2tAT3BlbkFJVaS0HiiZkQUSrD6Xcy9EfDm_f9MzTNl41BboyuxccA"

RESULTS_FILE = 'assets/botresults/results_smaldone_19:06.json' # TODO: Replace with your response file
client = OpenAI()

# answers_base = json.loads(Path('evaldata/all.json').read_text()) 
answers_base = pd.read_json('assets/evaldata/all.json')
answers_ours = pd.read_json(RESULTS_FILE)

def safe_filename(s):
    return "".join(x for x in s if x.isalnum())

output_file_name = f'assets/botresults/eval_smaldone_-{safe_filename(RESULTS_FILE)}.json'

In [38]:
answers_base

,input,language,id,prediction,source_file
0,How can businesses benefit from Swisscom’s sol...,EN,,Swisscom offers a comprehensive portfolio of s...,promts.json
1,What are the current promotions available on t...,EN,,Swisscom regularly announces interesting offer...,promts.json
2,How can users learn about Swisscom’s latest pr...,EN,,Users can stay updated about Swisscom's latest...,promts.json
3,Can users customize their experience on the Sw...,EN,,"Yes, users can customize their experience on t...",promts.json
4,Is there a search bar to help users find speci...,EN,,"I'm sorry, I can't help you with that right no...",promts.json
...,...,...,...,...,...
67,How do I set up swisscom with a new phone?,EN,,"To set up Swisscom with a new phone, you will ...",regular.json
68,Forgot my PIN. Help,EN,,"If you have forgotten the PIN for your SIM, pr...",regular.json
69,wifi nearly isn't working at all. Why is inter...,EN,,Solve your Internet issue with our intelligent...,regular.json
70,I paid my swisscom bill but the balance still ...,EN,,It may take up to 60 minutes for the payment s...,regular.json


In [39]:
answers_ours

,input,language,id,prediction,source_file
0,How can businesses benefit from Swisscom’s sol...,EN,,Swisscom offers a wide range of solutions tail...,promts.json
1,What are the current promotions available on t...,EN,,Swisscom often has exciting promotions across ...,promts.json
2,How can users learn about Swisscom’s latest pr...,EN,,"To stay updated on Swisscom's latest products,...",promts.json
3,Can users customize their experience on the Sw...,EN,,"Yes, users can customize their experience on t...",promts.json
4,Is there a search bar to help users find speci...,EN,,"Yes, the Swisscom website features a search ba...",promts.json
...,...,...,...,...,...
67,How do I set up swisscom with a new phone?,EN,,Setting up Swisscom on a new phone is straight...,regular.json
68,Forgot my PIN. Help,EN,,"If you've forgotten your PIN, don't worry! You...",regular.json
69,wifi nearly isn't working at all. Why is inter...,EN,,I'm sorry to hear you're experiencing slow int...,regular.json
70,I paid my swisscom bill but the balance still ...,EN,,I'm sorry to hear about the issue with your bi...,regular.json


In [40]:
class Decision(BaseModel):
    result: Literal['A','B','Tie']
    short_reason: str

def evaluate_responses(question, response_a, response_b) -> Decision:
    messages = [
        {"role": "system", "content": """
You are an LLM judge evaluating two chatbot responses to the same question. Please follow these criteria:
1. **Language consistency**: Does the chatbot answer in the language of the user? Does it adapt if the user asks to switch languages?
2. **Answer consistency**: Does the response accurately follow from any prior conversation context? Does it handle unrelated questions appropriately?
3. **Relevance**: Is the answer inherently related to the question?
4. **Factuality**: Is the response based on the correct and relevant information?
5. **Correlation**: Does the answer mention relevant references, if applicable?

For each question, choose the better response. Announce the result as A, B or Tie.
"""},
        {"role": "user", "content": f"""
Question: "{question}"

Response A: {response_a}
Response B: {response_b}

Please decide which response better meets the criteria above.
        """}
    ]


    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=messages,
        response_format=Decision,
    )

    decision: Decision = completion.choices[0].message.parsed
    return decision

In [41]:
from tqdm.cli import tqdm


# Main evaluation loop
results = []

their_records = answers_base.to_dict(orient='records')
our_records = answers_ours.to_dict(orient='records')

for idx, (entry_a, entry_b) in tqdm(enumerate(zip(their_records, our_records))):
    question = entry_a['input']
    response_a = entry_a['prediction']
    response_b = entry_b['prediction']
    
    # Get evaluation from the LLM
    decision = evaluate_responses(question, response_a, response_b)

    if decision.result == 'A':
        score = -1
    elif decision.result == 'B':
        score = +1
    elif decision.result == 'Tie':
        score = 0

    # Append result
    results.append({
        "question": question,
        "base": response_a,
        "ours": response_b,
        "reason": decision.short_reason,
        "score": score,
    })
    
    # Output results every time
    with open(output_file_name, 'w') as f:
        json.dump(results, f, indent=1)

print("Evaluation complete. Results saved to ", output_file_name)

72it [00:47,  1.51it/s]

Evaluation complete. Results saved to  assets/botresults/eval_smaldone_-assetsbotresultsresultssmaldone1906json.json


In [42]:
results = pd.read_json(output_file_name)

In [43]:
results.score.mean() # >0 means we win, <0 means they win and 0 means tied

0.7222222222222222